# Data
## Raw rainfall data Source:
- [University of Delaware Center for Climatic Research “Terrestrial Precipitation: 1900–2017 Gridded Monthly Time Series (1900–2017)](https://psl.noaa.gov/data/gridded/data.UDel_AirT_Precip.html)
- Monthly values for 1900/01 - 2017/12 V5.01
- Precipitation data information (OPeNDAP URL) is available on this [webpage](https://psl.noaa.gov/thredds/catalog/Datasets/udel.airt.precip/catalog.html?dataset=Datasets/udel.airt.precip/precip.mon.total.v501.nc)
- Each coordinate represents one IFLS district centroid.

## Processed Indonesian district coordinates data source:
- Maccini, Sharon, and Yang, Dean. Replication data for: Under the Weather: Health, Schooling, and Economic Consequences of Early-Life Rainfall. Nashville, TN: American Economic Association [publisher], 2009. Ann Arbor, MI: Inter-university Consortium for Political and Social Research [distributor], 2019-10-12. https://doi.org/10.3886/E113296V1

## Process flow (tentative):
1. Set up coordinates dataset
2. Fetch rainfall data through OPeNDAP URL and match it with district coordinates
3. Merge the coordinates and rainfall data
4.

## 1. Set up, fetch data from the link above and merge with disrict coordinates

In [2]:
# check working directory
import os
os.getcwd()
os.listdir()

['District_Indonesia.dta', 'rainfall.ipynb', 'README.md', '.git', '.idea']

In [ ]:
!pip -q install xarray netCDF4 pandas numpy

In [13]:
import pandas as pd
import numpy as np
import xarray as xr


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [78]:
# load district coordinates data
districts = pd.read_stata("data/external/District_Indonesia.dta")


#districts = districts.rename(columns={
#    "district": "district_id",
#    "lat": "latitude",
#    "lon": "longitude"
#}).filter(["district_id", "latitude", "longitude"], axis=1)
print(districts)
len(districts)

# show the max and min of longitude
max_lon = districts["longitude"].max()
min_lon = districts["longitude"].min()
print(f"maximum longitude:{max_lon}, minimum longitude:{min_lon}")

# show the max and min of latitude
max_lat = districts["latitude"].max()
min_lat = districts["latitude"].min()
print(f"maximum latitude:{max_lat}, minimum latitude:{min_lat}")

     province_ifls1  district_ifls1  province  district  district_name  \
0              11.0             5.0       NaN       NaN     aceh barat   
1              11.0             6.0       NaN       NaN     aceh besar   
2              11.0             1.0      11.0       3.0   aceh selatan   
3               NaN             NaN      11.0       2.0   aceh singkil   
4              11.0             4.0      11.0       6.0    aceh tengah   
..              ...             ...       ...       ...            ...   
338             NaN             NaN      18.0       7.0      way kanan   
339            33.0            12.0      33.0      12.0       wonogiri   
340            33.0             7.0      33.0       7.0       wonosobo   
341            82.0             8.0      92.0       4.0  yapen waropen   
342            34.0            71.0      34.0      71.0     yogyakarta   

     latitude   longitude  
0        4.50   96.000000  
1        5.50   95.419998  
2        3.00   97.250000  

In [79]:
# connect to rainfall dataset through opendap
url = "http://psl.noaa.gov/thredds/dodsC/Datasets/udel.airt.precip/precip.mon.total.v501.nc"
ds = xr.open_dataset(url, engine="netcdf4")
print(ds.data_vars)

Note:Caching=1


Data variables:
    precip   (time, lat, lon) float32 1GB ...


In [93]:
# variable name
var = "precip"
assert var in ds.data_vars, f"{var} not found; available: {list(ds.data_vars)}"

# 1) subset in space/time on the DATASET (keeps ds as a Dataset)
ds_sub = ds.sel(time=slice("1950-01-01","2000-12-31"),
                lon=slice(95, 140),
                lat=slice(6, -10))

# 2) pick the DATAARRAY for precipitation and materialize it once
da = ds_sub[var].load()
print(da)

# 3) district coordinate arrays (float)
lat_d = districts["latitude"].astype(float).to_numpy()
lon_d = districts["longitude"].astype(float).to_numpy()

# 4) sample nearest gridcell for ALL districts at once
p = da.sel(
    lon = xr.DataArray(lon_d, dims="district_id"),
    lat = xr.DataArray(lat_d, dims="district_id"),
    method="nearest"
)  # dims: time × district

print(p)


<xarray.DataArray 'precip' (time: 612, lat: 32, lon: 90)> Size: 7MB
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [23.210001 , 26.179998 , 25.42     , ...,        nan,
                nan,        nan],
        [       nan, 29.560001 , 29.16     , ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [22.7      , 25.48     , 28.62     , ...,        nan,
                nan,        nan],
        [       nan, 28.35     , 28.310001 , ...,        nan,
                nan,        nan],
...
        [       nan,        nan,        nan, ...,

In [94]:
out = p.to_dataframe("precip_cm").reset_index()
print(out.head(10))

        time  district_id     lon   lat  precip_cm
0 1950-01-01            0   96.25  4.75  29.160000
1 1950-01-01            1   95.25  5.75        NaN
2 1950-01-01            2   97.25  3.25  31.980000
3 1950-01-01            3   97.75  2.25        NaN
4 1950-01-01            4   97.25  3.75  26.060001
5 1950-01-01            5   97.75  3.75  25.870001
6 1950-01-01            6   97.75  4.75  25.700001
7 1950-01-01            7   96.75  5.25  22.030001
8 1950-01-01            8  136.25 -4.75        NaN
9 1950-01-01            9  136.75 -3.25  30.980000


In [96]:
# prepare for merging
ids = districts.reset_index().rename(columns={"index":"row_idx"})
ids["row_idx"] = np.arange(len(ids))

print(ids.head(10))

out = out.merge(
    ids[["row_idx","province_ifls1", "district_ifls1", "province", "district", "district_name"]],
    left_on="district_id", right_on="row_idx", how="left"
)

print(out.head(10))

   row_idx  province_ifls1  district_ifls1  province  district  district_name  \
0        0            11.0             5.0       NaN       NaN     aceh barat   
1        1            11.0             6.0       NaN       NaN     aceh besar   
2        2            11.0             1.0      11.0       3.0   aceh selatan   
3        3             NaN             NaN      11.0       2.0   aceh singkil   
4        4            11.0             4.0      11.0       6.0    aceh tengah   
5        5            11.0             2.0      11.0       4.0  aceh tenggara   
6        6            11.0             3.0      11.0       5.0     aceh timur   
7        7            11.0             8.0       NaN       NaN     aceh utara   
8        8             NaN             NaN      92.0       1.0    adm. mimika   
9        9             NaN             NaN      92.0       3.0    adm. paniai   

   latitude   longitude  
0      4.50   96.000000  
1      5.50   95.419998  
2      3.00   97.250000  
3   

In [97]:
# split time to year and month
out["year"]  = out["time"].dt.year
out["month"] = out["time"].dt.month
#out = out[["row_idx", "year", "month", "province_ifls1", "district_ifls1", "province", "district", "district_name", "precip_cm", "lat", "lon"]]

# rearrange
out = out[["year", "month", "province_ifls1", "district_ifls1", "province", "district", "district_name","precip_cm", "lat", "lon" ]]

print(out.head(10))


   year  month  province_ifls1  district_ifls1  province  district  \
0  1950      1            11.0             5.0       NaN       NaN   
1  1950      1            11.0             6.0       NaN       NaN   
2  1950      1            11.0             1.0      11.0       3.0   
3  1950      1             NaN             NaN      11.0       2.0   
4  1950      1            11.0             4.0      11.0       6.0   
5  1950      1            11.0             2.0      11.0       4.0   
6  1950      1            11.0             3.0      11.0       5.0   
7  1950      1            11.0             8.0       NaN       NaN   
8  1950      1             NaN             NaN      92.0       1.0   
9  1950      1             NaN             NaN      92.0       3.0   

   district_name  precip_cm   lat     lon  
0     aceh barat  29.160000  4.75   96.25  
1     aceh besar        NaN  5.75   95.25  
2   aceh selatan  31.980000  3.25   97.25  
3   aceh singkil        NaN  2.25   97.75  
4    aceh

In [101]:
out.to_csv("data/processed/precip_year_month.csv", index=False)

## 2. Trim the rainfall data

In [74]:
import pandas as pd

In [99]:
rf = pd.read_csv("data/processed/precip_year_month.csv")
print(rf.head(10))

   year  month  province_ifls1  district_ifls1  province  district  \
0  1950      1            11.0             5.0       NaN       NaN   
1  1950      1            11.0             6.0       NaN       NaN   
2  1950      1            11.0             1.0      11.0       3.0   
3  1950      1             NaN             NaN      11.0       2.0   
4  1950      1            11.0             4.0      11.0       6.0   
5  1950      1            11.0             2.0      11.0       4.0   
6  1950      1            11.0             3.0      11.0       5.0   
7  1950      1            11.0             8.0       NaN       NaN   
8  1950      1             NaN             NaN      92.0       1.0   
9  1950      1             NaN             NaN      92.0       3.0   

   district_name  precip_cm   lat     lon  
0     aceh barat  29.160000  4.75   96.25  
1     aceh besar        NaN  5.75   95.25  
2   aceh selatan  31.980000  3.25   97.25  
3   aceh singkil        NaN  2.25   97.75  
4    aceh